In [3]:
# Import Modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('dark_background')

%load_ext autoreload
%autoreload 2

pd.set_option('display.min_row', 25)
pd.set_option('display.max_column', 100)
pd.set_option('display.max_colwidth', 300)

### Possible additional datasets:

- Current Land Use Zoning Detail: https://data.seattle.gov/Land-Base/Current-Land-Use-Zoning-Detail/9nvb-wk9b
- Bike Rack Locations (Contains lat&long): https://data.seattle.gov/Land-Base/Bike-Racks/pbej-cxb2
- Marked Crosswalks (Contains various, including at signal or stop sign): https://data.seattle.gov/Land-Base/Marked-Crosswalks/dx75-5pzj
- Seattle Light Poles Location (Contains lat&long): https://data.seattle.gov/Land-Base/Seattle-City-Light-Poles/f4y8-37gx
- Traffic Signals (Contains if there is a bike signal or not): https://data.seattle.gov/Land-Base/Traffic-Signals/s63a-bkj8
- Traffic Circles (Contains lat&long): https://data.seattle.gov/Land-Base/Traffic-Circles/hw9f-j7b8
- Radar Speed Signs (Contains lat&long): https://data.seattle.gov/Land-Base/Radar-Speed-Signs/siht-4gsh

##### Unrelated: 
- Areaways: https://data.seattle.gov/Land-Base/Areaways/nmja-kgz6

In [4]:
collisions = pd.read_csv('../data/Collisions.csv')

In [5]:
streets = pd.read_csv('../data/Seattle_Streets.csv')

In [6]:
collisions.head()

,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,-122.362907,47.558372,1,326120,327620,3824232,Unmatched,Block,NaN,DELRIDGE WAY SW BETWEEN SW EDMUNDS ST AND SW HUDSON ST,,NaN,1,Property Damage Only Collision,NaN,1,0,0,0,0,0,0,2019/11/16 00:00:00+00,11/16/2019,Mid-Block (not related to intersection),16.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE SIDESWIPE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,N
1,-122.301868,47.618931,2,65200,65200,2621299,Matched,Block,NaN,E DENNY WAY BETWEEN 23RD AVE AND 24TH AVE,NaN,NaN,1,Property Damage Only Collision,Parked Car,2,0,0,2,0,0,0,2007/01/15 00:00:00+00,1/15/2007 4:25:00 PM,Mid-Block (not related to intersection),12.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, RIGHT SIDE AT ANGLE",NaN,0,Clear,Ice,Dusk,NaN,7015014.0,NaN,32,One parked--one moving,0,0,N
2,-122.391508,47.669461,3,20100,20100,1473746,Matched,Block,NaN,NW 56TH ST BETWEEN 26TH AVE NW AND 28TH AVE NW,NaN,NaN,1,Property Damage Only Collision,Parked Car,2,0,0,2,0,0,0,2004/03/20 00:00:00+00,3/20/2004 10:25:00 PM,Driveway Junction,11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END AT ANGLE",NaN,0,Clear,Dry,Dark - Street Lights On,NaN,4080013.0,NaN,32,One parked--one moving,0,0,N
3,-122.346296,47.620903,4,49300,49300,2072241,Matched,Intersection,28927.0,TAYLOR AVE N AND THOMAS ST,NaN,NaN,2,Injury Collision,Angles,4,0,0,2,3,0,0,2006/06/03 00:00:00+00,6/3/2006,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END AT ANGLE",NaN,0,Clear,Dry,Daylight,NaN,6154013.0,NaN,10,Entering at angle,0,0,N
4,-122.288688,47.532714,5,29800,29800,1177964,Unmatched,Block,NaN,BEACON ER AVE S BETWEEN S PORTLAND ST AND S CHICAGO ST,NaN,NaN,0,Unknown,NaN,0,0,0,0,0,0,0,2004/11/10 00:00:00+00,11/10/2004,Mid-Block (not related to intersection),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END AT ANGLE",NaN,NaN,NaN,NaN,NaN,NaN,4315006.0,NaN,NaN,NaN,0,0,N


In [27]:
collisions['SEVERITYDESC'].value_counts()

Property Damage Only Collision    136964
Injury Collision                   58441
Unknown                            21510
Serious Injury Collision            3077
Fatality Collision                   346
Name: SEVERITYDESC, dtype: int64

In [7]:
streets.head()

,OBJECTID,ARTCLASS,COMPKEY,UNITID,UNITID2,UNITIDSORT,UNITDESC,STNAME_ORD,XSTRLO,XSTRHI,ARTDESCRIPT,OWNER,STATUS,BLOCKNBR,SPEEDLIMIT,SEGDIR,ONEWAY,ONEWAYDIR,FLOW,SEGLENGTH,SURFACEWIDTH,SURFACETYPE_1,SURFACETYPE_2,INTRLO,DIRLO,INTKEYLO,INTRHI,DIRHI,NATIONHWYSYS,STREETTYPE,PVMTCONDINDX1,PVMTCONDINDX2,TRANCLASS,TRANDESCRIPT,SLOPE_PCT,PVMTCATEGORY,PARKBOULEVARD,SHAPE_Length
0,1,2.0,1006,10.0,120.0,100120.0,1ST AVE BETWEEN SENECA ST AND UNIVERSITY ST,1ST AVE,SENECA ST,UNIVERSITY ST,Minor Arterial,,INSVC,1200.0,25.0,NW,N,,,306.0,48.0,PCC,AC/PCC,1ST AVE AND SENECA ST,NW,29611.0,1ST AVE AND UNIVERSITY ST,SE,N,Downtown Neighborhood,87.0,62.0,1,PRINCIPAL TRANSIT ROUTE,4.0,ART,N,305.966050
1,2,2.0,1009,10.0,150.0,100150.0,1ST AVE BETWEEN PIKE ST AND PINE ST,1ST AVE,PIKE ST,PINE ST,Minor Arterial,,INSVC,1500.0,25.0,NW,N,,,426.0,104.0,AC/PCC,PCC,1ST AVE AND PIKE ST,NW,29593.0,1ST AVE AND PINE ST,SE,N,Downtown Neighborhood,57.0,58.0,1,PRINCIPAL TRANSIT ROUTE,5.0,ART,N,426.031562
2,3,0.0,1032,15.0,80.0,150080.0,1ST AVE N BETWEEN VALLEY UPPER ST AND ALOHA ST,1ST AVE N,VALLEY UPPER ST,ALOHA ST,Not Designated,,INSVC,800.0,20.0,N,N,,,297.0,0.0,PCC,NaN,1ST AVE N AND VALLEY UPPER ST,N,28897.0,1ST AVE N AND ALOHA ST,S,N,Neighborhood Yield Street,0.0,0.0,0,NOT DESIGNATED,17.0,NON-ART,N,297.147592
3,4,0.0,1051,15.0,230.0,150230.0,1ST AVE N BETWEEN LYNN ST AND MCGRAW S ST,1ST AVE N,LYNN ST,MCGRAW S ST,Not Designated,,INSVC,2200.0,20.0,N,N,,,175.0,25.0,AC,NaN,1ST AVE N AND LYNN ST,N,28113.0,1ST AVE N AND MCGRAW S ST,S,N,Neighborhood Yield Street,9.0,0.0,0,NOT DESIGNATED,3.0,NON-ART,N,174.804983
4,5,0.0,1060,15.0,282.0,150282.0,1ST AVE N BETWEEN FULTON S ST AND FULTON N ST,1ST AVE N,FULTON S ST,FULTON N ST,Not Designated,,INSVC,2800.0,20.0,N,N,,,73.0,0.0,PCC,NaN,1ST AVE N AND FULTON S ST,N,28051.0,1ST AVE N AND FULTON N ST,S,N,Neighborhood Yield Street,0.0,0.0,0,NOT DESIGNATED,5.0,NON-ART,N,73.110708


In [8]:
bikeracks = pd.read_csv('../data/Bike_Racks.csv')
bikeracks.head()

,X,Y,OBJECTID,COMPKEY,COMPTYPE,SEGKEY,DISTANCE,WIDTH,UNITID,UNITTYPE,UNITDESC,BIKE_FACILITY,RACK_CAPACITY,DELINEATOR_POSTS,FINISH_TYPE,SURFACE_TYPE,INSTALL_DATE,MOUNT_TYPE,COLOR,MANUFACTURER,MODEL_TYPE,CONDITION,CONDITION_ASSESSMENT_DATE,SIDE,OWNERSHIP,CURRENT_STATUS,PRIMARYDISTRICTCD,SECONDARYDISTRICTCD,OVERRIDEYN,OVERRIDECOMMENT,SHAPE_LNG,SHAPE_LAT
0,1.257066e+06,247817.450418,1,639535,16,19236,427.0,-31.0,BKR-3650,BKR,NW MARKET ST 0240 BLOCK N SIDE ( 402) 402 FT W/O 24TH AVE NW,SGL,2,N,GALV,PCC,2016/02/29 00:00:00+00,,,,,GOOD,NaN,N,SDOT,INSVC,DISTRICT6,,N,NaN,-122.389310,47.668757
1,1.278455e+06,227063.233437,2,591433,16,14902,125.0,29.0,BKR-3190,BKR,E UNION ST 0240 BLOCK N SIDE ( 112) 112 FT E/O 24TH AVE,CLSTR,6,N,GALV,PCC,2012/11/30 00:00:00+00,SUFMT,,SDOT,BKR-RLRCK,GOOD,2012/10/20 00:00:00+00,N,SDOT,PLANNED,DISTRICT3,,N,NaN,-122.300910,47.613022
2,1.275207e+06,250005.982260,3,518834,16,17262,370.0,-29.0,BKR-520,BKR,NE 65TH ST 0100 BLOCK S SIDE ( 355) 39 FT W/O 12TH AVE NE,SGL,2,N,GALV,PCC,2000/11/12 00:00:00+00,SUFMT,NONE,BKR-CRMTL,BKR-RLRCK,GOOD,2009/07/30 00:00:00+00,S,SDOT,INSVC,DISTRICT4,,N,NaN,-122.315866,47.675737
3,1.271784e+06,228002.570526,4,519092,16,12225,206.0,17.0,BKR-422,BKR,PINE ST 0120 BLOCK NW SIDE ( 193) 11 FT SW/O MELROSE AVE,SGL,2,N,GALV,PCC,2005/05/18 00:00:00+00,SUFMT,NONE,BKR-CRMTL,BKR-RLRCK,GOOD,NaN,NW,SDOT,INSVC,DISTRICT3,,N,NaN,-122.328026,47.615245
4,1.263540e+06,253266.784759,5,517282,16,18846,37.0,-17.0,BKR-2280,BKR,NW 76TH ST 0060 BLOCK N SIDE ( 24) 24 FT W/O 6TH AVE NW,SGL,4,N,PAINT,PCC,NaN,SUFMT,BLK,BKR-CORA,BKR-W2704,GOOD,2009/07/31 00:00:00+00,N,SDOT,INSVC,DISTRICT6,,N,NaN,-122.363480,47.684049


In [9]:
bikeracks['UNITTYPE'].value_counts()

BKR    3953
Name: UNITTYPE, dtype: int64

In [17]:
bikeracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3953 entries, 0 to 3952
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   X                          3940 non-null   float64
 1   Y                          3940 non-null   float64
 2   OBJECTID                   3953 non-null   int64  
 3   COMPKEY                    3953 non-null   int64  
 4   COMPTYPE                   3953 non-null   int64  
 5   SEGKEY                     3953 non-null   int64  
 6   DISTANCE                   3953 non-null   float64
 7   WIDTH                      3946 non-null   float64
 8   UNITID                     3953 non-null   object 
 9   UNITTYPE                   3953 non-null   object 
 10  UNITDESC                   3953 non-null   object 
 11  BIKE_FACILITY              3953 non-null   object 
 12  RACK_CAPACITY              3953 non-null   int64  
 13  DELINEATOR_POSTS           3953 non-null   objec

In [15]:
radar = pd.read_csv('../data/Radar_Speed_Signs.csv')
radar.head()

,X,Y,OBJECTID,COMPKEY,UNITID,UNITDESC,OWNERSHIP,CONDITION,CURRENT_STATUS,OVERRIDEYN,COMPTYPE,SEGKEY,UNITTYPE,BLUE_TOOTH_ENABLED,DATA_COLLECTION_CAPABILITY,SERVICE_POINT,SOLAR_OPERATED,SIDE,SHAPE_LNG,SHAPE_LAT
0,1.260736e+06,257337.873980,1,508865,RSS-1,HOLMAN RD NW 0907 BLOCK NW SIDE ( 110) 110 FT NE/O MARY AVE NW,SDOT,GOOD,INSVC,N,16,10938,RSS,N,N,NaN,Y,NW,-122.375191,47.695054
1,1.276266e+06,258974.658109,2,535585,RSS-20,15TH AVE NE 0980 BLOCK W SIDE ( 0) 0 FT /O NE 100TH ST,SDOT,GOOD,PLANNED,N,16,2072,RSS,Y,Y,NaN,Y,W,-122.312268,47.700376
2,1.291550e+06,189642.035446,3,702898,RSS-51,RAINIER AVE S 1010 BLOCK N SIDE ( 355) 1069 FT W/O 75TH AVE S,,,INSVC,N,16,12438,RSS,N,N,NaN,Y,N,-122.245047,47.511122
3,1.282505e+06,244565.359086,4,678320,RSS-49,NE 45TH ST 0380 BLOCK S SIDE ( 315) 235 FT W/O 40TH AVE NE,,,,N,16,16866,RSS,N,N,NaN,N,S,-122.285830,47.661206
4,1.262416e+06,209772.973835,5,600918,RSS-36,SW GENESEE ST 0250 BLOCK N SIDE ( 10) 10 FT W/O 25TH AVE SW,,,INSVC,N,16,22200,RSS,N,N,NaN,N,N,-122.364529,47.564773


In [16]:
radar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   X                           50 non-null     float64
 1   Y                           50 non-null     float64
 2   OBJECTID                    51 non-null     int64  
 3   COMPKEY                     51 non-null     int64  
 4   UNITID                      51 non-null     object 
 5   UNITDESC                    51 non-null     object 
 6   OWNERSHIP                   51 non-null     object 
 7   CONDITION                   51 non-null     object 
 8   CURRENT_STATUS              51 non-null     object 
 9   OVERRIDEYN                  51 non-null     object 
 10  COMPTYPE                    51 non-null     int64  
 11  SEGKEY                      51 non-null     int64  
 12  UNITTYPE                    51 non-null     object 
 13  BLUE_TOOTH_ENABLED          51 non-nu

In [22]:
traf_circles = pd.read_csv('../data/Traffic_Circles.csv')
traf_circles.head()

,X,Y,OBJECTID,COMPKEY,UNITID,UNITDESC,CONDITION,CONDITION_ASSESSMENT_DATE,CURRENT_STATUS,OWNERSHIP,PRIMARYDISTRICTCD,SECONDARYDISTRICTCD,COMPTYPE,CURRENT_STATUS_DATE,INSTALL_DATE,INSTALL_PURPOSE,INTKEY,LANDSCAPING,MAINTAINED_BY,MAINTENANCE_AGREEMENT,NO_LAND_REASON,OVERRIDECOMMENT,OVERRIDEYN,OWNERSHIP_DATE,SDOT_INITIATED,SEWER_ACCESS_COVER,TRCSIZE,SURVEY_MONUMENT,TRCSHAPE,UNITTYPE,SHAPE_LNG,SHAPE_LAT
0,1.258588e+06,210475.502412,1,509338,TRC-601,38TH AVE SW AND SW DAKOTA ST,GOOD,2009/09/15 00:00:00+00,INSVC,SDOT,DISTRICT1,,16,2009/10/14 00:00:00+00,2000/05/16 00:00:00+00,,31889,Y,SDOT,NaN,,NaN,N,1970/01/01 00:00:00+00,N,N,0,N,CRC,TRC,-122.380088,47.566489
1,1.266983e+06,260074.185342,2,509566,TRC-934,FREMONT AVE N AND N 102ND ST,GOOD,2009/09/09 00:00:00+00,INSVC,SDOT,DISTRICT5,,16,2009/10/14 00:00:00+00,1996/01/02 00:00:00+00,,37392,Y,SDOT,NaN,,NaN,N,1970/01/01 00:00:00+00,N,N,0,N,CRC,TRC,-122.350050,47.702894
2,1.274207e+06,254314.130296,3,508937,TRC-784,8TH AVE NE AND NE 81ST ST,GOOD,2009/08/03 00:00:00+00,INSVC,SDOT,DISTRICT4,,16,2009/10/14 00:00:00+00,1991/03/20 00:00:00+00,,24389,Y,SDOT,NaN,,NaN,N,1970/01/01 00:00:00+00,N,N,16,N,CRC,TRC,-122.320263,47.687493
3,1.278437e+06,217535.101459,4,509329,TRC-422,25TH AVE S AND S GRAND ST,GOOD,2009/09/21 00:00:00+00,INSVC,SDOT,DISTRICT3,,16,2009/10/14 00:00:00+00,2003/02/10 00:00:00+00,,31416,Y,SDOT,NaN,,NaN,N,1970/01/01 00:00:00+00,N,N,0,N,OTHER,TRC,-122.300247,47.586905
4,1.280848e+06,255438.451194,5,509639,TRC-542,32ND AVE NE AND NE 86TH ST,GOOD,2009/09/15 00:00:00+00,INSVC,SDOT,DISTRICT5,,16,2009/10/14 00:00:00+00,1999/06/29 00:00:00+00,,37684,Y,SDOT,NaN,,NaN,N,1970/01/01 00:00:00+00,N,N,0,N,CRC,TRC,-122.293392,47.690923


In [23]:
traf_circles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1066 entries, 0 to 1065
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   X                          1066 non-null   float64
 1   Y                          1066 non-null   float64
 2   OBJECTID                   1066 non-null   int64  
 3   COMPKEY                    1066 non-null   int64  
 4   UNITID                     1066 non-null   object 
 5   UNITDESC                   1066 non-null   object 
 6   CONDITION                  1066 non-null   object 
 7   CONDITION_ASSESSMENT_DATE  1066 non-null   object 
 8   CURRENT_STATUS             1066 non-null   object 
 9   OWNERSHIP                  1066 non-null   object 
 10  PRIMARYDISTRICTCD          1066 non-null   object 
 11  SECONDARYDISTRICTCD        1066 non-null   object 
 12  COMPTYPE                   1066 non-null   int64  
 13  CURRENT_STATUS_DATE        1066 non-null   objec

In [20]:
traf_signals = pd.read_csv('../data/Traffic_Signals.csv')
traf_signals.head()

,OBJECTID,COMPKEY,COMPTYPE,UNITID,DESCRIPTION,UNITDESC,SEGKEY,INTKEY,INT_UNITID,SIGNAL_TYPE,SIGNAL_MAINT_DIST,INSTALL_DATE,ADDDTTM,MODDTTM,OWNERSHIP,CONDITION,CONDITION_ASSESSMENT_DATE,CURRENT_STATUS,CURRENT_STATUS_DATE,ARTERIAL_CLASS,MAINT_AGREE,MAINT_BY,INT_SIGNAL_TYPE_CD,PRIMARYDISTRICTCD,SECONDARYDISTRICTCD,PEDAUDIODEVICEYN,PEDSIGNALYN,PH_MODELTYPE,PP_MODELTYPE,PP_QUANTITY,PEDHDFIRSTINSTALLDT,PH_QUANTITY,PEDAUDIOINSTALLDT,BIKESIGNALHDYN,ASBLT,HALFSIGNALYN,METEREDYN,LTTURNSIGNALYN,LTPROTECTEDPERMYN,LTPROTECTEDYN,RTTURNSIGNALYN,RTPROTECTEDPERMYN,RTPROTECTEDYN,LTFIRSTINSTALLDT,LTREMOVEDT,RTFIRSTINSTALLDT,RTREMOVEDT,DETDEVSTOPBARYN,DETDEVADVANCEDYN,DETDEVSYSTEMYN,CABMODELTYPECD,COMCONNTYPECD,MMUMODELTYPECD,PE_MODELTYPE,PI_MODELTYPE,INSTALLERCD,JOINTOWNERSHIPYN,CO_MODELTYPE,MMUMANCD,MMUINSTALLDT,LASTSIGNALOPTDT,PEDPSHINSTALLDT,NUM_ATTACHMENTS
0,1,272753,74,SGL-278,0.0,GREENWOOD AVE N AND N 80TH ST,1,24282,4521,SEMI,NORTH,1970/01/01 00:00:00+00,2005/03/17 17:11:06+00,2019/02/15 00:00:00+00,SDOT,GOOD,2008/03/06 00:00:00+00,INSVC,2008/12/15 00:00:00+00,2.0,NaN,,CITY,DISTRICT6,,N,N,LEDCD,NaN,NaN,1970/01/01 00:00:00+00,8,1970/01/01 00:00:00+00,N,NaN,N,N,Y,Y,N,N,N,N,2011/10/06 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,Y,N,N,3TS1,,NSM-12,NaN,NaN,,N,"LC8000, LMD9200",EDI,2019/02/12 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,0
1,2,272754,74,SGL-669,0.0,7TH AVE AND WESTLAKE AVE,1,29542,39139,PRE,DOWNTOWN,1970/01/01 00:00:00+00,2005/03/17 17:11:06+00,2020/04/01 00:00:00+00,SDOT,GOOD,2008/09/18 00:00:00+00,INSVC,2008/12/15 00:00:00+00,1.0,NaN,,CITY,DISTRICT7,,N,N,LEDCD,NaN,NaN,1970/01/01 00:00:00+00,8,1970/01/01 00:00:00+00,N,NaN,N,N,N,N,N,N,N,N,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,N,N,N,2,COPPER,NSM-12,OPT,NaN,,Y,"M34, , M52, M60-ATC",EDI,2020/03/05 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,0
2,3,272755,74,SGL-178,0.0,2ND AVE AND BROAD ST,1,29666,39521,PRE,DOWNTOWN,1970/01/01 00:00:00+00,2005/03/17 17:11:06+00,2020/04/06 00:00:00+00,SDOT,POOR,2008/08/29 00:00:00+00,INSVC,2008/12/15 00:00:00+00,1.0,NaN,,CITY,DISTRICT7,,N,N,LEDCD,NAV-2W-PPB,8,1970/01/01 00:00:00+00,8,1970/01/01 00:00:00+00,N,NaN,N,N,N,N,N,N,N,N,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,N,N,N,3TS2-Type1,FIBER,MMU-16LEip,NaN,NaN,,Y,"M34, M60",EDI,2020/04/04 00:00:00+00,1970/01/01 00:00:00+00,2018/01/11 00:00:00+00,0
3,5,272757,74,SGL-022,0.0,DEARBORN OFF RP AND S DEARBORN ST,1,30444,44306,SEMI,SOUTH,1970/01/01 00:00:00+00,2005/03/17 17:11:07+00,1970/01/01 00:00:00+00,WSDOT,,1970/01/01 00:00:00+00,INSVC,2008/12/15 00:00:00+00,1.0,NaN,,STATE,DISTRICT2,,N,N,NaN,NaN,NaN,1970/01/01 00:00:00+00,NaN,1970/01/01 00:00:00+00,N,NaN,N,N,N,N,N,N,N,N,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,N,N,N,,,,NaN,NaN,,NaN,NaN,,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,0
4,6,272758,74,SGL-75,0.0,4TH AVE AND PIKE ST,1,29589,39315,PRE,DOWNTOWN,1970/01/01 00:00:00+00,2005/03/17 17:11:07+00,2019/07/22 00:00:00+00,SDOT,FAIR,2008/09/08 00:00:00+00,INSVC,2008/12/15 00:00:00+00,1.0,NaN,,CITY,DISTRICT7,,N,N,"LED, LEDCD",NaN,NaN,1970/01/01 00:00:00+00,"8, 8",1970/01/01 00:00:00+00,N,NaN,N,N,N,N,N,N,N,N,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,N,N,N,3TS2,COPPER,MMU2-16LEi,NaN,NaN,,N,"M34, , , M52, M52",EDI,2019/07/15 00:00:00+00,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,0


In [21]:
# no lat&long location but has district and intersection
# could check out the dates installed
traf_signals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1113 entries, 0 to 1112
Data columns (total 63 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   OBJECTID                   1113 non-null   int64  
 1   COMPKEY                    1113 non-null   int64  
 2   COMPTYPE                   1113 non-null   int64  
 3   UNITID                     1113 non-null   object 
 4   DESCRIPTION                1058 non-null   float64
 5   UNITDESC                   1113 non-null   object 
 6   SEGKEY                     1113 non-null   int64  
 7   INTKEY                     1113 non-null   int64  
 8   INT_UNITID                 1113 non-null   object 
 9   SIGNAL_TYPE                1113 non-null   object 
 10  SIGNAL_MAINT_DIST          1113 non-null   object 
 11  INSTALL_DATE               1113 non-null   object 
 12  ADDDTTM                    1113 non-null   object 
 13  MODDTTM                    1113 non-null   objec

In [10]:
marked_cross = pd.read_csv('../data/Marked_Crosswalks.csv')
marked_cross.head()

,X,Y,OBJECTID,COMPKEY,UNITID,UNITDESC,CONDITION,CONDITION_ASSESSMENT_DATE,OWNERSHIP,CURRENT_STATUS,PRIMARYDISTRICTCD,SECONDARYDISTRICTCD,OVERRIDEYN,COMPTYPE,SEGKEY,UNITTYPE,OLD_ID,APPROACH,MARKING_TYPE,SCHOOL,MIDBLOCK_CROSSWALK,INSTALL_DATE,COLOR,COMMENTS,CATEGORY,OWNERSHIP_DATE,CURRENT_STATUS_DATE,MAINTAINED_BY,MAINTENANCE_AGREEMENT,CURBSPACEID,MAINT_DISTRICT,OVERRIDECOMMENT,SHAPE_LNG,SHAPE_LAT,CROSSWALK_CONTROL,MATERIAL,ONSTREET,XSTRLOW,MEAS_FROM_LOW,DISTTOLOW,STPOINT,XSTRHI,MEAS_FROM_HI,DISTTOHI,OFFSET,SIDE,MEASUREMENT_ORIGIN,ATTACHMENT_1,ATTACHMENT_2,ATTACHMENT_3,ATTACHMENT_4,ATTACHMENT_5,ATTACHMENT_6,ATTACHMENT_7,ATTACHMENT_8,ATTACHMENT_9,MAINTENANCE_GROUP,NUM_ATTACHMENTS
0,1.269560e+06,223140.574842,1,522399,XWK-3325,ALASKAN WAY 0060 BLOCK C SIDE ( 35) 35 FT NW/O YESLER WAY,GOOD,2012/09/30 00:00:00+00,,INSVC,DISTRICT7,,N,16,8717,XWK,Old ID # 758,NW,LADER,N,N,1970/01/01 00:00:00+00,WHT,"ALASKAN WAY & YESLER WAY, N",XWK,1970/01/01 00:00:00+00,2009/01/22 00:00:00+00,,NaN,NaN,CENTRAL,NaN,-122.336656,47.601800,,THRPL,ALASKAN WAY,YESLER WAY,Y,35,0,COLUMBIA ST,N,234,0,C,XStreet Centerline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XWK-CD7-YEAR4,0
1,1.283979e+06,201215.071449,2,525064,XWK-3029,S HOLLY ST 0440 BLOCK C SIDE ( 385) 20 FT W/O 45TH AVE S,GOOD,2013/08/26 00:00:00+00,,INSVC,DISTRICT2,,N,16,20363,XWK,Old ID # 662,W,LADER,Y,N,1970/01/01 00:00:00+00,WHT,"WI #13959 8/27/98 <br> School # 8<br>45 AVE S & S HOLLY ST, W",XWK,1970/01/01 00:00:00+00,2008/11/14 00:00:00+00,,NaN,NaN,SOUTH,NaN,-122.276553,47.542459,,THRPL,S HOLLY ST,44TH AVE S,N,385,0,45TH AVE S,Y,20,0,C,XStreet Centerline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XWK-CD2-YEAR4,0
2,1.268498e+06,242364.672876,3,523429,XWK-416,STONE WAY N 0390 BLOCK C SIDE ( 115) 115 FT N/O N 39TH ST,GOOD,2017/06/29 00:00:00+00,SDOT,INSVC,DISTRICT4,,N,16,12959,XWK,Old ID # 485,S,LADER,N,N,1970/01/01 00:00:00+00,WHT,"<p>WI #18482 <br>STONE WAY N &amp; BRIDGE WAY N, S</p><p><br></p><p>Remarked per WO 528218</p>",XWK,1970/01/01 00:00:00+00,2009/02/10 00:00:00+00,SDOT,NaN,NaN,NORTH,NaN,-122.342486,47.654435,SIGNAL,THRPL,STONE WAY N,N 39TH ST,Y,115,0,BRIDGE WAY N,N,83,0,C,XStreet Centerline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XWK-CD4-YEAR1,0
3,1.259573e+06,211119.144160,4,521447,XWK-2387,35TH AVE SW 0390 BLOCK C SIDE ( 18) 18 FT S/O SW ANDOVER N ST,GOOD,2019/07/23 00:00:00+00,SDOT,INSVC,DISTRICT1,,N,16,5446,XWK,Old ID # 162,S,LADER,Y,N,1970/01/01 00:00:00+00,WHT,"School # 74<br>35 AVE SW & SW ANDOVER ST (W LEG), S",XWK,1970/01/01 00:00:00+00,2009/01/06 00:00:00+00,,NaN,NaN,SOUTH,NaN,-122.376151,47.568308,,THRPL,35TH AVE SW,SW ANDOVER N ST,Y,18,0,SW ANDOVER S ST,N,112,0,C,XStreet Centerline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XWK-CD1-YEAR3,0
4,1.277718e+06,271174.016176,5,524665,XWK-1892,NE 145TH ST 0200 BLOCK C SIDE ( 5) 5 FT E/O 20TH AVE NE,GOOD,2012/06/15 00:00:00+00,,INSVC,DISTRICT5,,N,16,16688,XWK,Old ID # 505,N,LADER,N,N,1970/01/01 00:00:00+00,WHT,"WI #18482 <br>City Limits<br>20 AVE NE & NE 145 ST, N",XWK,1970/01/01 00:00:00+00,2012/12/13 00:00:00+00,,NaN,NaN,NORTH,NaN,-122.307324,47.733890,,THRPL,NE 145TH ST,20TH AVE NE,Y,5,0,22ND AVE NE,N,328,0,C,XStreet Centerline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XWK-CD5-YEAR4,0


In [12]:
marked_cross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5683 entries, 0 to 5682
Data columns (total 58 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   X                          5670 non-null   float64
 1   Y                          5670 non-null   float64
 2   OBJECTID                   5683 non-null   int64  
 3   COMPKEY                    5683 non-null   int64  
 4   UNITID                     5683 non-null   object 
 5   UNITDESC                   5683 non-null   object 
 6   CONDITION                  5683 non-null   object 
 7   CONDITION_ASSESSMENT_DATE  5683 non-null   object 
 8   OWNERSHIP                  5683 non-null   object 
 9   CURRENT_STATUS             5683 non-null   object 
 10  PRIMARYDISTRICTCD          5683 non-null   object 
 11  SECONDARYDISTRICTCD        5683 non-null   object 
 12  OVERRIDEYN                 5683 non-null   object 
 13  COMPTYPE                   5683 non-null   int64

## Folium Map

In [14]:
import folium
from geopy.geocoders import Nominatim

In [15]:
# Grab general lat&long for Seattle, WA to have a center for map
sea_address = 'Seattle, WA'

sea_geolocator = Nominatim(user_agent='foursquare_agent')
sea_location = sea_geolocator.geocode(sea_address)
sea_latitude = sea_location.latitude
sea_longitude = sea_location.longitude
print('Seattle coordinates: ', sea_latitude, sea_longitude)

Seattle coordinates:  47.6038321 -122.3300624


In [24]:
# #Generate map centered around address
seattle_map = folium.Map(location=[sea_latitude, sea_longitude], zoom_start=13)

#red marker added for main location
folium.features.CircleMarker(
    [sea_latitude,sea_longitude],
    radius=10,
    color='black',
    popup='Seattle',
    fill=True,
    fill_color= 'black',
    fill_opacity = 0.6
).add_to(seattle_map)

#blue markers for marked crosswalks
for lat, lng, label in zip(marked_cross['SHAPE_LAT'][:100], marked_cross['SHAPE_LNG'][:100], 'CROSSWALKS'):
    folium.features.CircleMarker(
        [lat, lng],
        radius = 5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(seattle_map)
    
#orange markers for bike racks
for lat, lng, label in zip(bikeracks['SHAPE_LAT'][:100], bikeracks['SHAPE_LNG'][:100], 'BIKE RACKS'):
    folium.features.CircleMarker(
        [lat, lng],
        radius = 5,
        color='orange',
        popup=label,
        fill=True,
        fill_color='orange',
        fill_opacity=0.6
    ).add_to(seattle_map)
    
#purple markers for traffic circles
for lat, lng, label in zip(traf_circles['SHAPE_LAT'][:100], traf_circles['SHAPE_LNG'][:100], 'TRAF CIR'):
    folium.features.CircleMarker(
        [lat, lng],
        radius = 5,
        color='purple',
        popup=label,
        fill=True,
        fill_color='purple',
        fill_opacity=0.6
    ).add_to(seattle_map)
    
#red markers for traffic circles
for lat, lng, label in zip(collisions['Y'][:100], collisions['X'][:100], collisions['SEVERITYDESC'][:100]):
    folium.features.CircleMarker(
        [lat, lng],
        radius = 5,
        color='red',
        popup=label,
        fill=True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(seattle_map)
    
seattle_map